## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-22-18-47-21 +0000,nyt,"Mexico Confirms Death of ‘El Mencho,’ Leader o...",https://www.nytimes.com/2026/02/22/world/ameri...
1,2026-02-22-18-45-10 +0000,nypost,"Mexican resort town, airport under siege after...",https://nypost.com/2026/02/22/world-news/mexic...
2,2026-02-22-18-44-04 +0000,nyt,Russia Attacks Ukraine Ahead of Invasion’s 4th...,https://www.nytimes.com/2026/02/22/world/europ...
3,2026-02-22-18-40-41 +0000,startribune,U.S. defeats Canada 2-1 in overtime to win men...,https://www.startribune.com/2026-winter-olympi...
4,2026-02-22-18-34-43 +0000,nyt,Trump Tensions Turn U.S.-Canada Olympic Hockey...,https://www.nytimes.com/2026/02/22/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2470/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
34,trump,38
7,new,9
80,iran,9
94,tariff,9
70,killed,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
52,2026-02-22-14-09-27 +0000,cbc,"Armed man shot, killed after entering perimete...",https://www.cbc.ca/news/world/trump-mar-a-lago...,91
51,2026-02-22-14-10-32 +0000,nypost,Secret Service kills shotgun-wielding man at T...,https://nypost.com/2026/02/22/us-news/secret-s...,89
24,2026-02-22-17-32-34 +0000,bbc,Armed man killed after entering secure perimet...,https://www.bbc.com/news/articles/cp32l9w9pgyo...,77
150,2026-02-21-23-57-00 +0000,wsj,President Trump announced a 15% global tariff ...,https://www.wsj.com/economy/trade/what-to-know...,73
113,2026-02-22-06-32-45 +0000,nypost,Trump reveals his ‘new hero’ Supreme Court jus...,https://nypost.com/2026/02/22/us-news/trump-ca...,71


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
52,91,2026-02-22-14-09-27 +0000,cbc,"Armed man shot, killed after entering perimete...",https://www.cbc.ca/news/world/trump-mar-a-lago...
119,40,2026-02-22-05-10-25 +0000,nypost,Homeland Security suspends TSA PreCheck and Gl...,https://nypost.com/2026/02/22/us-news/homeland...
122,39,2026-02-22-04-47-09 +0000,nypost,Iran president says Tehran won’t ‘bow’ as war ...,https://nypost.com/2026/02/21/world-news/iran-...
113,33,2026-02-22-06-32-45 +0000,nypost,Trump reveals his ‘new hero’ Supreme Court jus...,https://nypost.com/2026/02/22/us-news/trump-ca...
144,31,2026-02-22-00-15-48 +0000,nypost,"Senior MP demands UK probe ex-Prince Andrew, P...",https://nypost.com/2026/02/21/world-news/senio...
164,28,2026-02-21-21-18-52 +0000,nypost,Panicked Dems fear ‘zombie campaigns’ will se...,https://nypost.com/2026/02/21/us-news/californ...
68,28,2026-02-22-12-56-38 +0000,nypost,NYC area set for 2 feet of snow in first dange...,https://nypost.com/2026/02/22/us-news/nyc-area...
128,27,2026-02-22-03-17-28 +0000,cbc,"Greenland PM declines Trump's hospital ship, u...",https://www.cbc.ca/news/world/trump-hospital-s...
55,22,2026-02-22-14-00-40 +0000,bbc,Iran students stage first large anti-governmen...,https://www.bbc.com/news/articles/c5yj2kzkrj0o...
1,21,2026-02-22-18-45-10 +0000,nypost,"Mexican resort town, airport under siege after...",https://nypost.com/2026/02/22/world-news/mexic...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
